In [3]:
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers 
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, Activation, MaxPooling2D, Dropout, Flatten

import tensorflow_addons as tfa
import cv2
import os 
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import datetime

from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
# from sklearn.metrics import accuracy_score
import random 
import time

AUTOTUNE = tf.data.AUTOTUNE


In [4]:
N_BEF_RES = 256
N_RES = 256 
# N_CLASSES = 143 
N_BATCH = 32 

input_shape = (N_RES, N_RES, 3)

In [22]:
# PATH = 'C:/Users/user/Desktop/Child Skin Disease'
PATH = "D:/Dropbox/2. WORK/SNUH/Child Skin Disease"
dataset = os.path.join(PATH, 'Total_Dataset')
dataset


'D:/Dropbox/2. WORK/SNUH/Child Skin Disease\\Total_Dataset'

In [23]:
# Train & test set
limit_num = 0
base_num = 100
train_dict = {}
test_dict = {} 

for i in range(6):
    files = os.listdir(os.path.join(dataset, f'H{i}'))
    
    for f in files: 
        imgs = glob(os.path.join(dataset, f'H{i}', f) + '/*.jpg')
        
        if len(imgs) > limit_num: 
            train_dict[f] = len(imgs)
            
for i in range(7, 10): 
    files = [val for val in list(train_dict.keys())]
    
    for f in files:
        imgs = glob(os.path.join(dataset, f'H{i}', f) + '/*.jpg')
        test_dict[f] = len(imgs) 
        

N_CLASSES = len(train_dict)


# train_dict, test_dict, N_CLASSES

In [24]:
train_images = [] 
test_images = []

for i in range(6):
    for key in train_dict.keys():
        img = glob(dataset + f'/H{str(i)}/{key}/*.jpg')
        train_images.extend(img) 
        
for i in range(7, 10):
    for key in train_dict.keys():
        img = glob(dataset + f'/H{str(i)}/{key}/*.jpg')
        test_images.extend(img) 
        
        
random.shuffle(train_images)
random.shuffle(test_images)
        
len(train_images), len(test_images)
        

(57558, 51092)

In [25]:
label_to_index = {}
for idx, key in zip(range(len(train_dict)), train_dict.keys()):
    label_to_index[key] = idx
    
label_to_index

{'Abscess': 0,
 'Acanthosis nigricans': 1,
 'Acne': 2,
 'Acne neonatorum': 3,
 'Acne scar integrated': 4,
 'Acquired tufted hamangioma': 5,
 'Alopecia areata': 6,
 'Anetoderma': 7,
 'Angioedema': 8,
 'Angiofibroma': 9,
 'Angiokeratoma': 10,
 'Angular cheilitis': 11,
 'Aplasia cutis, congenital': 12,
 "Beau's lines": 13,
 "Becker's nevus": 14,
 'Blue nevus': 15,
 'Cafe-au-lait spot': 16,
 'Cellulitis': 17,
 'Cheilitis': 18,
 'Chicken pox (varicella)': 19,
 'Childhood granulomatous periorificial dermatitis': 20,
 'Confluent and reticulated papillomatosis': 21,
 'Congenital Hemangioma': 22,
 'Contact dermatitis': 23,
 'Corn, Callus': 24,
 'Cutaneous larva migrans': 25,
 'Cutaneous lupus erythematosus': 26,
 'Cutis marmorata': 27,
 'Cutis marmorata telangiectatica congenita (CMTC)': 28,
 'Cyst integrated': 29,
 'Dermal Melanocytic Hamartoma': 30,
 'Dermatofibroma': 31,
 'Drug eruption': 32,
 'Dyshidrotic eczema': 33,
 'Eczema herpeticum': 34,
 'Epidermal nevus': 35,
 'Erythema dyschromicum

In [26]:
def train_skin_data(files):
    
    for file in files:
    
        f = file.decode('utf-8')
        
        img = cv2.imread(f, cv2.COLOR_BGR2YCR_CB)
        img = cv2.resize(img, (N_BEF_RES, N_BEF_RES))
        img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)

        idx = f.split('\\')[1].split('/')[2]
        lbl = tf.keras.utils.to_categorical(label_to_index[idx], len(train_dict))

        yield (img, lbl)    


In [27]:
def test_skin_data(files):
    
    for file in files:
    
        f = file.decode('utf-8')
        
        img = cv2.imread(f, cv2.COLOR_BGR2YCR_CB)
        img = cv2.resize(img, (N_BEF_RES, N_BEF_RES))
        img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
        
        lbl = tf.keras.utils.to_categorical(label_to_index[f.split('\\')[1].split('/')[2]], len(train_dict))

        yield (img, lbl)    

In [28]:
# to adapt dataset to model 
train = [] 

for f in train_images: 
    # print(f)
    img = cv2.imread(f, cv2.COLOR_BGR2YCR_CB)
    img = cv2.resize(img, (N_BEF_RES, N_BEF_RES))
    img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
    
    train.append(img) 
    
x_train = tf.reshape(train, [-1, N_RES, N_RES, 3])

D:/Dropbox/2. WORK/SNUH/Child Skin Disease\Total_Dataset/H1/Ulcer\H1_218424_P3_L0.jpg
D:/Dropbox/2. WORK/SNUH/Child Skin Disease\Total_Dataset/H3/Alopecia areata\H3_2410_P3_L0.jpg
D:/Dropbox/2. WORK/SNUH/Child Skin Disease\Total_Dataset/H1/Cafe-au-lait spot\H1_98004_P4_L0.jpg
D:/Dropbox/2. WORK/SNUH/Child Skin Disease\Total_Dataset/H1/Psoriasis\H1_200066_P14_L0.jpg
D:/Dropbox/2. WORK/SNUH/Child Skin Disease\Total_Dataset/H0/Psoriasis\H0_112583_P1_L0.jpg
D:/Dropbox/2. WORK/SNUH/Child Skin Disease\Total_Dataset/H0/Melanocytic nevus\H0_196909_P2_L2.jpg
D:/Dropbox/2. WORK/SNUH/Child Skin Disease\Total_Dataset/H0/Cyst integrated\H0_8377_P4_L0.jpg
D:/Dropbox/2. WORK/SNUH/Child Skin Disease\Total_Dataset/H1/Blue nevus\H1_211002_P3_L0.jpg
D:/Dropbox/2. WORK/SNUH/Child Skin Disease\Total_Dataset/H1/Ichthyosis\H10_3441_P1_L0.jpg
D:/Dropbox/2. WORK/SNUH/Child Skin Disease\Total_Dataset/H1/Normal\H1_92819_P1_L1.jpg
D:/Dropbox/2. WORK/SNUH/Child Skin Disease\Total_Dataset/H1/Psoriasis\H1_98600_P5_L

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.02),
        layers.RandomWidth(0.2),
        layers.RandomHeight(0.2),
    ]
)

# Setting the state of the normalization layer.
data_augmentation.layers[0].adapt(x_train)

In [ ]:
def create_encoder():
    resnet = keras.applications.ResNet50V2(
        include_top=False, weights=None, input_shape=input_shape, pooling="avg"
    )

    inputs = keras.Input(shape=input_shape)
    augmented = data_augmentation(inputs)
    outputs = resnet(augmented)
    model = keras.Model(inputs=inputs, outputs=outputs, name="cifar10-encoder")
    return model


encoder = create_encoder()
encoder.summary()

learning_rate = 0.001
batch_size = 265
hidden_units = 512
projection_units = 128
num_epochs = 50
dropout_rate = 0.5
temperature = 0.05

In [ ]:
def create_classifier(encoder, trainable=True):
    
    for layer in encoder.layers:
        layer.trainable = trainable

    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    features = layers.Dropout(dropout_rate)(features)
    features = layers.Dense(hidden_units, activation="relu")(features)
    features = layers.Dropout(dropout_rate)(features)
    outputs = layers.Dense(N_CLASSES, activation="softmax")(features)

    model = keras.Model(inputs=inputs, outputs=outputs, name="cifar10-classifier")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )
    return model

In [ ]:
train_dataset = tf.data.Dataset.from_generator(train_skin_data, 
                                               output_types=(tf.float64, tf.float32), 
                                               output_shapes=(tf.TensorShape([N_BEF_RES, N_BEF_RES, 3]), tf.TensorShape([N_CLASSES])),
                                               args=[train_images])


test_dataset = tf.data.Dataset.from_generator(test_skin_data, 
                                              output_types=(tf.float64, tf.float32), 
                                              output_shapes=(tf.TensorShape([N_BEF_RES, N_BEF_RES, 3]), tf.TensorShape([N_CLASSES])),
                                              args=[test_images])

In [ ]:
split_size = int(len(train_images) * 0.2)
split_train_dataset = train_dataset.skip(split_size)
split_val_dataset = train_dataset.take(split_size)

In [ ]:
encoder = create_encoder()
classifier = create_classifier(encoder)
classifier.summary()

with tf.device('/device:GPU:0'):
    hist = classifier.fit(split_train_dataset, 
                          batch_size=batch_size,  
                          epochs=num_epochs, validation_data=split_val_dataset)


In [ ]:

accuracy = classifier.evaluate(test_dataset)
print(f"Test accuracy: {round(accuracy * 100, 2)}%")